** Sentiment analysis of tweets with Python, NLTK, Word2vec and Scikit-learn **

http://zablo.net/blog/post/twitter-sentiment-analysis-python-scikit-word2vec-nltk-xgboost

This jupyter notebook describes an example of sentiment analysis for twitter posts. Sentiment analysis can predict many different emotions attached to the text, but in this report only 3 major were considered: positive, negative and neutral.

The training dataset was small (just over 5900 examples) and the data within it was highly skewed, which greatly impacted on the difficulty of building good classifier. 

** Data was pre-processed using pandas, gensim and numpy libraries and the learning/validating process was built with scikit-learn. **

Plots were created using plotly.



In [8]:
from collections import Counter
import nltk
import pandas as pd
#from emoticons import EmoticonDetector
import re as regex
import numpy as np
#import plotly
#from plotly import graph_objs
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, RandomizedSearchCV
from time import time
import gensim

# plotly configuration
#plotly.offline.init_notebook_mode()

In [5]:
# A class named "TwitterData_Initialize" is defined.
# class: a logical grouping of data and functions.
class TwitterData_Initialize():
    data = []
    processed_data = []
    wordlist = []

    data_model = None
    data_labels = None
    is_testing = False
    
    # A function (method) named "initialize" is defined    
    def initialize(self, csv_file, is_testing_set=False, from_cached=None):
        if from_cached is not None:
            self.data_model = pd.read_csv(from_cached)
            return

        self.is_testing = is_testing_set

        if not is_testing_set:
            # If no header exists, use "header=None"
            # By using "header=0", one can replace existing headers
            self.data = pd.read_csv(csv_file, header=0, names=["id", "emotion", "text"])
            # only select emotions that are pos, neg, or neu.
            self.data = self.data[self.data["emotion"].isin(["positive", "negative", "neutral"])]
        else:
            self.data = pd.read_csv(csv_file, header=0, names=["id", "text"],dtype={"id":"int64","text":"str"},nrows=4000)
            not_null_text = 1 ^ pd.isnull(self.data["text"])
            not_null_id = 1 ^ pd.isnull(self.data["id"])
            self.data = self.data.loc[not_null_id & not_null_text, :]

        self.processed_data = self.data
        self.wordlist = []
        self.data_model = None
        self.data_labels = None

In [6]:
# create a new object (instance) named data
# For example, I can have data1 = TwitterData_Initialize(), 
#data2 = TwitterData_Initialize(), and data3 = TwitterData_Initialize().
# self represents data1, data2, and data3
data = TwitterData_Initialize()
data.initialize("Data/twitter_train.csv")
data.processed_data.head(5)

,id,emotion,text
0,635769805279248384,negative,Not Available
1,635930169241374720,neutral,IOS 9 App Transport Security. Mm need to check...
2,635950258682523648,neutral,"Mar if you have an iOS device, you should down..."
3,636030803433009153,negative,@jimmie_vanagon my phone does not run on lates...
4,636100906224848896,positive,Not sure how to start your publication on iOS?...


In [11]:
df = data.processed_data
neg = len(df[df["emotion"] == "negative"])
pos = len(df[df["emotion"] == "positive"])
neu = len(df[df["emotion"] == "neutral"])

print "neg, pos and neu counts are:", neg, pos, neu

neg, pos and neu counts are: 956 2888 2125


** Preprocessing **

1 Cleansing

     Remove URLs
     Remove usernames (mentions)
     Remove tweets with *Not Available* text
     Remove special characters
     Remove numbers

2 Text processing

     Tokenize
     Transform to lowercase
     Stem

3 Build word list for Bag-of-Words

** 1. Cleasing **

use **regular expression** to clearn twitter posts. 

In [12]:
class TwitterCleanuper:
    def iterate(self):
        for cleanup_method in [self.remove_urls,
                               self.remove_usernames,
                               self.remove_na,
                               self.remove_special_chars,
                               self.remove_numbers]:
            yield cleanup_method

    @staticmethod
    def remove_by_regex(tweets, regexp):
        tweets.loc[:, "text"].replace(regexp, "", inplace=True)
        return tweets

    def remove_urls(self, tweets):
        # regex.compile: compile a regular expression pattern into a regular expression object
        # '\s' matches whitespace character
        #'^' matches start of a string
        #'[]' indicates a set of characters, [^\s] means start of a string or a whitespace
        return TwitterCleanuper.remove_by_regex(tweets, regex.compile(r"http.?://[^\s]+[\s]?"))

    def remove_na(self, tweets):
        return tweets[tweets["text"] != "Not Available"]

    def remove_special_chars(self, tweets):  # it unrolls the hashtags to normal words
        for remove in map(lambda r: regex.compile(regex.escape(r)), [",", ":", "\"", "=", "&", ";", "%", "$",
                                                                     "@", "%", "^", "*", "(", ")", "{", "}",
                                                                     "[", "]", "|", "/", "\\", ">", "<", "-",
                                                                     "!", "?", ".", "'",
                                                                     "--", "---", "#"]):
            tweets.loc[:, "text"].replace(remove, "", inplace=True)
        return tweets

    def remove_usernames(self, tweets):
        return TwitterCleanuper.remove_by_regex(tweets, regex.compile(r"@[^\s]+[\s]?"))

    def remove_numbers(self, tweets):
        return TwitterCleanuper.remove_by_regex(tweets, regex.compile(r"\s?[0-9]+\.?[0-9]*"))

In [13]:
class TwitterData_Cleansing(TwitterData_Initialize):
    def __init__(self, previous):
        self.processed_data = previous.processed_data
        
    def cleanup(self, cleanuper):
        t = self.processed_data
        for cleanup_method in cleanuper.iterate():
            if not self.is_testing:
                t = cleanup_method(t)
            else:
                if cleanup_method.__name__ != "remove_na":
                    t = cleanup_method(t)

        self.processed_data = t

In [14]:
data = TwitterData_Cleansing(data)
data.cleanup(TwitterCleanuper())
data.processed_data.head(5)

/home/tqi/anaconda2/lib/python2.7/site-packages/pandas/core/generic.py:3813: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



,id,emotion,text
1,635930169241374720,neutral,IOS App Transport Security Mm need to check if...
2,635950258682523648,neutral,Mar if you have an iOS device you should downl...
3,636030803433009153,negative,my phone does not run on latest IOS which may ...
4,636100906224848896,positive,Not sure how to start your publication on iOS ...
5,636176272947744772,neutral,Two Dollar Tuesday is here with Forklift Quick...


** Tokenization & stemming **